In [13]:
import numpy as np
import argparse
import imutils
import time
import cv2
import os

#Put own path to file here 
video = "castro.mov"
conf = 0.5
threshold = 0.1

classname = []
list_of_vehicles = ["car","truck"]
def get_vehicle_count(boxes, class_names):
	total_vehicle_count = 0 # total vechiles present in the image
	dict_vehicle_count = {} # dictionary with count of each distinct vehicles detected
	for i in range(len(boxes)):
		class_name = class_names[i]
		# print(i,".",class_name)
		if(class_name in list_of_vehicles):
			total_vehicle_count += 1
			dict_vehicle_count[class_name] = dict_vehicle_count.get(class_name,0) + 1

	return total_vehicle_count, dict_vehicle_count


ped = ["person"]
def get_ped_count(boxes, class_names):
	total_ped_count = 0 # total vechiles present in the image
	dict_ped_count = {} # dictionary with count of each distinct vehicles detected
	for i in range(len(boxes)):
		class_name = class_names[i]
		# print(i,".",class_name)
		if(class_name in ped):
			total_ped_count += 1
			dict_ped_count[class_name] = dict_ped_count.get(class_name,0) + 1

	return total_ped_count, dict_ped_count

#Own Path here too
# load the COCO class labels our YOLO model was trained on
labelsPath = "coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

#Own Path here too
# derive the paths to the YOLO weights and model configuration
weightsPath = "yolov3.weights"
configPath = "yolov3.cfg"
# load our YOLO object detector trained on COCO dataset (80 classes)
# and determine only the *output* layer names that we need from YOLO
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(video)
writer = None
(W, H) = (None, None)

# try to determine the total number of frames in the video file
try:
	prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
		else cv2.CAP_PROP_FRAME_COUNT
	total = int(vs.get(prop))
	print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
	print("[INFO] could not determine # of frames in video")
	print("[INFO] no approx. completion time can be provided")
	total = -1

# loop over frames from the video file stream


framecount = 0
totalvech = 0
totalped = 0
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break

	# if the frame dimensions are empty, grab them
	if W is None or H is None:
		(H, W) = frame.shape[:2]

	# construct a blob from the input frame and then perform a forward
	# pass of the YOLO object detector, giving us our bounding boxes
	# and associated probabilities
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	net.setInput(blob)
	start = time.time()
	layerOutputs = net.forward(ln)
	end = time.time()

	# initialize our lists of detected bounding boxes, confidences,
	# and class IDs, respectively
	boxes = []
	confidences = []
	classIDs = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence (i.e., probability)
			# of the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# filter out weak predictions by ensuring the detected
			# probability is greater than the minimum probability
			if confidence > conf:
				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# use the center (x, y)-coordinates to derive the top
				# and and left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates,
				# confidences, and class IDs
				boxes.append([x, y, int(width), int(height)])
				confidences.append(float(confidence))
				classIDs.append(classID)
				classname.append(LABELS[classID])

	# apply non-maxima suppression to suppress weak, overlapping
	# bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, conf,threshold)

	# ensure at least one detection exists
	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# draw a bounding box rectangle and label on the frame
			color = [int(c) for c in COLORS[classIDs[i]]]
			cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
			text = "{}: {:.4f}".format(LABELS[classIDs[i]],
				confidences[i])
			cv2.putText(frame, text, (x, y - 5),
				cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

	# check if the video writer is None

	total_vehicles,vech = get_vehicle_count(boxes, classname)
	print("Total vechicles in frame  " + str(framecount) +" : ",total_vehicles)
	total_ped,ped = get_ped_count(boxes,classname) 
	print("Total pedestrians in frame  " + str(framecount) +" : ",total_ped)
	font = cv2.FONT_HERSHEY_SIMPLEX    
	cv2.putText(frame,"Pedestrian Count: " + str(total_ped),(50, 50),font, 1,(0, 255, 255),  2, cv2.LINE_4)  
	cv2.putText(frame,"Vehicle Count: " + str(total_vehicles),(20, 20),font, 1,(0, 220, 220),  2, cv2.LINE_4) 
	framecount = int(framecount) + 1
	totalvech += total_vehicles
	totalped += total_ped
	#print(totalped)  
	if writer is None:
		# initialize our video writer. Put file type in for *""
		fourcc = cv2.VideoWriter_fourcc(*"MOVV")
		writer = cv2.VideoWriter("tester.mov", fourcc, 30,
			(frame.shape[1], frame.shape[0]), True)

		# some information on processing single frame
		if total > 0:
			elap = (end - start)
			workTimes = elap * total
			workHours = workTimes / 3600 
			print("[INFO] every one frame took {:.4f} seconds".format(elap))
			print("[INFO] estimated total time to finish process: {:.2f} minutes".format(
				workTimes / 60))

	# write the output frame to disk
	writer.write(frame)

print("Done!")
writer.release()
vs.release()

[INFO] loading YOLO from disk...
[INFO] 3918 total frames in video
Total vechicles in frame  0 :  8
Total pedestrians in frame  0 :  5
[INFO] every one frame took 0.4608 seconds
[INFO] estimated total time to finish process: 30.09 minutes
Total vechicles in frame  1 :  8
Total pedestrians in frame  1 :  5
Total vechicles in frame  2 :  8
Total pedestrians in frame  2 :  5
Total vechicles in frame  3 :  8
Total pedestrians in frame  3 :  5
Total vechicles in frame  4 :  8
Total pedestrians in frame  4 :  5
Total vechicles in frame  5 :  12
Total pedestrians in frame  5 :  6
Total vechicles in frame  6 :  12
Total pedestrians in frame  6 :  6
Total vechicles in frame  7 :  14
Total pedestrians in frame  7 :  6
Total vechicles in frame  8 :  15
Total pedestrians in frame  8 :  6
Total vechicles in frame  9 :  16
Total pedestrians in frame  9 :  6
Total vechicles in frame  10 :  11
Total pedestrians in frame  10 :  6
Total vechicles in frame  11 :  11
Total pedestrians in frame  11 :  6
To

Total vechicles in frame  114 :  10
Total pedestrians in frame  114 :  5
Total vechicles in frame  115 :  11
Total pedestrians in frame  115 :  5
Total vechicles in frame  116 :  11
Total pedestrians in frame  116 :  5
Total vechicles in frame  117 :  9
Total pedestrians in frame  117 :  5
Total vechicles in frame  118 :  9
Total pedestrians in frame  118 :  5
Total vechicles in frame  119 :  9
Total pedestrians in frame  119 :  5
Total vechicles in frame  120 :  9
Total pedestrians in frame  120 :  5
Total vechicles in frame  121 :  8
Total pedestrians in frame  121 :  5
Total vechicles in frame  122 :  8
Total pedestrians in frame  122 :  5
Total vechicles in frame  123 :  8
Total pedestrians in frame  123 :  5
Total vechicles in frame  124 :  8
Total pedestrians in frame  124 :  5
Total vechicles in frame  125 :  8
Total pedestrians in frame  125 :  5
Total vechicles in frame  126 :  8
Total pedestrians in frame  126 :  5
Total vechicles in frame  127 :  8
Total pedestrians in frame

Total vechicles in frame  228 :  8
Total pedestrians in frame  228 :  5
Total vechicles in frame  229 :  8
Total pedestrians in frame  229 :  5
Total vechicles in frame  230 :  8
Total pedestrians in frame  230 :  5
Total vechicles in frame  231 :  8
Total pedestrians in frame  231 :  5
Total vechicles in frame  232 :  8
Total pedestrians in frame  232 :  5
Total vechicles in frame  233 :  8
Total pedestrians in frame  233 :  5
Total vechicles in frame  234 :  8
Total pedestrians in frame  234 :  5
Total vechicles in frame  235 :  8
Total pedestrians in frame  235 :  5
Total vechicles in frame  236 :  8
Total pedestrians in frame  236 :  5
Total vechicles in frame  237 :  8
Total pedestrians in frame  237 :  5
Total vechicles in frame  238 :  8
Total pedestrians in frame  238 :  5
Total vechicles in frame  239 :  8
Total pedestrians in frame  239 :  5
Total vechicles in frame  240 :  8
Total pedestrians in frame  240 :  5
Total vechicles in frame  241 :  8
Total pedestrians in frame  2

Total vechicles in frame  341 :  16
Total pedestrians in frame  341 :  10
Total vechicles in frame  342 :  16
Total pedestrians in frame  342 :  10
Total vechicles in frame  343 :  16
Total pedestrians in frame  343 :  10
Total vechicles in frame  344 :  21
Total pedestrians in frame  344 :  10
Total vechicles in frame  345 :  19
Total pedestrians in frame  345 :  10
Total vechicles in frame  346 :  20
Total pedestrians in frame  346 :  10
Total vechicles in frame  347 :  20
Total pedestrians in frame  347 :  10
Total vechicles in frame  348 :  20
Total pedestrians in frame  348 :  10
Total vechicles in frame  349 :  20
Total pedestrians in frame  349 :  10
Total vechicles in frame  350 :  16
Total pedestrians in frame  350 :  10
Total vechicles in frame  351 :  16
Total pedestrians in frame  351 :  10
Total vechicles in frame  352 :  16
Total pedestrians in frame  352 :  10
Total vechicles in frame  353 :  16
Total pedestrians in frame  353 :  10
Total vechicles in frame  354 :  16
To

Total vechicles in frame  453 :  16
Total pedestrians in frame  453 :  10
Total vechicles in frame  454 :  16
Total pedestrians in frame  454 :  10
Total vechicles in frame  455 :  16
Total pedestrians in frame  455 :  10
Total vechicles in frame  456 :  16
Total pedestrians in frame  456 :  10
Total vechicles in frame  457 :  16
Total pedestrians in frame  457 :  10
Total vechicles in frame  458 :  16
Total pedestrians in frame  458 :  10
Total vechicles in frame  459 :  16
Total pedestrians in frame  459 :  10
Total vechicles in frame  460 :  16
Total pedestrians in frame  460 :  10
Total vechicles in frame  461 :  16
Total pedestrians in frame  461 :  10
Total vechicles in frame  462 :  16
Total pedestrians in frame  462 :  10
Total vechicles in frame  463 :  16
Total pedestrians in frame  463 :  10
Total vechicles in frame  464 :  16
Total pedestrians in frame  464 :  10
Total vechicles in frame  465 :  16
Total pedestrians in frame  465 :  10
Total vechicles in frame  466 :  16
To

...

Total vechicles in frame  3449 :  3
Total pedestrians in frame  3449 :  0
Total vechicles in frame  3450 :  3
Total pedestrians in frame  3450 :  0
Total vechicles in frame  3451 :  3
Total pedestrians in frame  3451 :  0
Total vechicles in frame  3452 :  3
Total pedestrians in frame  3452 :  0
Total vechicles in frame  3453 :  3
Total pedestrians in frame  3453 :  0
Total vechicles in frame  3454 :  3
Total pedestrians in frame  3454 :  0
Total vechicles in frame  3455 :  3
Total pedestrians in frame  3455 :  0
Total vechicles in frame  3456 :  3
Total pedestrians in frame  3456 :  0
Total vechicles in frame  3457 :  3
Total pedestrians in frame  3457 :  0
Total vechicles in frame  3458 :  3
Total pedestrians in frame  3458 :  0
Total vechicles in frame  3459 :  3
Total pedestrians in frame  3459 :  0
Total vechicles in frame  3460 :  3
Total pedestrians in frame  3460 :  0
Total vechicles in frame  3461 :  3
Total pedestrians in frame  3461 :  0
Total vechicles in frame  3462 :  3
To

Total vechicles in frame  3560 :  4
Total pedestrians in frame  3560 :  0
Total vechicles in frame  3561 :  4
Total pedestrians in frame  3561 :  0
Total vechicles in frame  3562 :  4
Total pedestrians in frame  3562 :  0
Total vechicles in frame  3563 :  4
Total pedestrians in frame  3563 :  0
Total vechicles in frame  3564 :  4
Total pedestrians in frame  3564 :  0
Total vechicles in frame  3565 :  4
Total pedestrians in frame  3565 :  0
Total vechicles in frame  3566 :  4
Total pedestrians in frame  3566 :  0
Total vechicles in frame  3567 :  5
Total pedestrians in frame  3567 :  0
Total vechicles in frame  3568 :  5
Total pedestrians in frame  3568 :  0
Total vechicles in frame  3569 :  4
Total pedestrians in frame  3569 :  0
Total vechicles in frame  3570 :  4
Total pedestrians in frame  3570 :  0
Total vechicles in frame  3571 :  4
Total pedestrians in frame  3571 :  0
Total vechicles in frame  3572 :  4
Total pedestrians in frame  3572 :  0
Total vechicles in frame  3573 :  3
To

Total vechicles in frame  3671 :  3
Total pedestrians in frame  3671 :  0
Total vechicles in frame  3672 :  3
Total pedestrians in frame  3672 :  0
Total vechicles in frame  3673 :  3
Total pedestrians in frame  3673 :  0
Total vechicles in frame  3674 :  3
Total pedestrians in frame  3674 :  0
Total vechicles in frame  3675 :  3
Total pedestrians in frame  3675 :  0
Total vechicles in frame  3676 :  3
Total pedestrians in frame  3676 :  0
Total vechicles in frame  3677 :  3
Total pedestrians in frame  3677 :  0
Total vechicles in frame  3678 :  3
Total pedestrians in frame  3678 :  0
Total vechicles in frame  3679 :  3
Total pedestrians in frame  3679 :  0
Total vechicles in frame  3680 :  3
Total pedestrians in frame  3680 :  0
Total vechicles in frame  3681 :  3
Total pedestrians in frame  3681 :  0
Total vechicles in frame  3682 :  3
Total pedestrians in frame  3682 :  0
Total vechicles in frame  3683 :  3
Total pedestrians in frame  3683 :  0
Total vechicles in frame  3684 :  3
To

Total vechicles in frame  3782 :  3
Total pedestrians in frame  3782 :  0
Total vechicles in frame  3783 :  3
Total pedestrians in frame  3783 :  0
Total vechicles in frame  3784 :  3
Total pedestrians in frame  3784 :  0
Total vechicles in frame  3785 :  3
Total pedestrians in frame  3785 :  0
Total vechicles in frame  3786 :  3
Total pedestrians in frame  3786 :  0
Total vechicles in frame  3787 :  3
Total pedestrians in frame  3787 :  0
Total vechicles in frame  3788 :  3
Total pedestrians in frame  3788 :  0
Total vechicles in frame  3789 :  3
Total pedestrians in frame  3789 :  0
Total vechicles in frame  3790 :  3
Total pedestrians in frame  3790 :  0
Total vechicles in frame  3791 :  3
Total pedestrians in frame  3791 :  0
Total vechicles in frame  3792 :  6
Total pedestrians in frame  3792 :  0
Total vechicles in frame  3793 :  6
Total pedestrians in frame  3793 :  0
Total vechicles in frame  3794 :  6
Total pedestrians in frame  3794 :  0
Total vechicles in frame  3795 :  6
To

Total vechicles in frame  3893 :  4
Total pedestrians in frame  3893 :  0
Total vechicles in frame  3894 :  4
Total pedestrians in frame  3894 :  0
Total vechicles in frame  3895 :  4
Total pedestrians in frame  3895 :  0
Total vechicles in frame  3896 :  4
Total pedestrians in frame  3896 :  0
Total vechicles in frame  3897 :  5
Total pedestrians in frame  3897 :  0
Total vechicles in frame  3898 :  5
Total pedestrians in frame  3898 :  0
Total vechicles in frame  3899 :  5
Total pedestrians in frame  3899 :  0
Total vechicles in frame  3900 :  5
Total pedestrians in frame  3900 :  0
Total vechicles in frame  3901 :  5
Total pedestrians in frame  3901 :  0
Total vechicles in frame  3902 :  5
Total pedestrians in frame  3902 :  0
Total vechicles in frame  3903 :  6
Total pedestrians in frame  3903 :  0
Total vechicles in frame  3904 :  6
Total pedestrians in frame  3904 :  0
Total vechicles in frame  3905 :  7
Total pedestrians in frame  3905 :  0
Total vechicles in frame  3906 :  7
To